In [1]:
import ast
import import_ipynb
import Extract_Python_Pairs as EPP
#https://docs.python.org/3/library/tokenize.html
import tokenize
import io
import numpy as np
from tqdm import tqdm #inline progress bar (quality of life)

importing Jupyter notebook from Extract_Python_Pairs.ipynb


In [3]:
###get the code and comment snippets from the Extract Python Pairs file
code_snippets, comments = EPP.get_python_pairs(EPP.get_all_data())

  0%|          | 58717/16115540 [00:00<00:27, 587128.57it/s]

Completed in 179 seconds


100%|██████████| 16115540/16115540 [00:20<00:00, 798420.75it/s] 

Total pairs: 2508330


In [4]:
'''
ENDMARKER = 0
NAME = 1
NUMBER = 2
STRING = 3
NEWLINE = 4
INDENT = 5
DEDENT = 6
LPAR = 7
RPAR = 8
LSQB = 9
RSQB = 10
COLON = 11
COMMA = 12
SEMI = 13
PLUS = 14
MINUS = 15
STAR = 16
SLASH = 17
VBAR = 18
AMPER = 19
LESS = 20
GREATER = 21
EQUAL = 22
DOT = 23
PERCENT = 24
LBRACE = 25
RBRACE = 26
EQEQUAL = 27
NOTEQUAL = 28
LESSEQUAL = 29
GREATEREQUAL = 30
TILDE = 31
CIRCUMFLEX = 32
LEFTSHIFT = 33
RIGHTSHIFT = 34
DOUBLESTAR = 35
PLUSEQUAL = 36
MINEQUAL = 37
STAREQUAL = 38
SLASHEQUAL = 39
PERCENTEQUAL = 40
AMPEREQUAL = 41
VBAREQUAL = 42
CIRCUMFLEXEQUAL = 43
LEFTSHIFTEQUAL = 44
RIGHTSHIFTEQUAL = 45
DOUBLESTAREQUAL = 46
DOUBLESLASH = 47
DOUBLESLASHEQUAL = 48
AT = 49
ATEQUAL = 50
RARROW = 51
ELLIPSIS = 52
COLONEQUAL = 53
OP = 54
AWAIT = 55
ASYNC = 56
TYPE_IGNORE = 57
TYPE_COMMENT = 58
# These aren't used by the C tokenizer but are needed for tokenize.py
ERRORTOKEN = 59
COMMENT = 60
NL = 61
ENCODING = 62
N_TOKENS = 63
# Special definitions for cooperation with parser
NT_OFFSET = 256
'''

"\nENDMARKER = 0\nNAME = 1\nNUMBER = 2\nSTRING = 3\nNEWLINE = 4\nINDENT = 5\nDEDENT = 6\nLPAR = 7\nRPAR = 8\nLSQB = 9\nRSQB = 10\nCOLON = 11\nCOMMA = 12\nSEMI = 13\nPLUS = 14\nMINUS = 15\nSTAR = 16\nSLASH = 17\nVBAR = 18\nAMPER = 19\nLESS = 20\nGREATER = 21\nEQUAL = 22\nDOT = 23\nPERCENT = 24\nLBRACE = 25\nRBRACE = 26\nEQEQUAL = 27\nNOTEQUAL = 28\nLESSEQUAL = 29\nGREATEREQUAL = 30\nTILDE = 31\nCIRCUMFLEX = 32\nLEFTSHIFT = 33\nRIGHTSHIFT = 34\nDOUBLESTAR = 35\nPLUSEQUAL = 36\nMINEQUAL = 37\nSTAREQUAL = 38\nSLASHEQUAL = 39\nPERCENTEQUAL = 40\nAMPEREQUAL = 41\nVBAREQUAL = 42\nCIRCUMFLEXEQUAL = 43\nLEFTSHIFTEQUAL = 44\nRIGHTSHIFTEQUAL = 45\nDOUBLESTAREQUAL = 46\nDOUBLESLASH = 47\nDOUBLESLASHEQUAL = 48\nAT = 49\nATEQUAL = 50\nRARROW = 51\nELLIPSIS = 52\nCOLONEQUAL = 53\nOP = 54\nAWAIT = 55\nASYNC = 56\nTYPE_IGNORE = 57\nTYPE_COMMENT = 58\n# These aren't used by the C tokenizer but are needed for tokenize.py\nERRORTOKEN = 59\nCOMMENT = 60\nNL = 61\nENCODING = 62\nN_TOKENS = 63\n# Special def

In [34]:
key_words = ['False','await','else','import','pass',
            'None','break','except','in','raise',
            'True','class','finally','is','return',
            'and','continue','for','lambda','try',
            'as','def','from','nonlocal','while',
            'assert','del','global','not','with',
            'async','elif','if','or','yield']

In [43]:
#https://docs.python.org/3/library/tokenize.html
def tokenize_python(code_snippet, genaraic_vars = True):
    tokens = tokenize.tokenize(io.BytesIO(code_snippet.encode('utf-8')).readline)
    parsed = []
    parsed.append('<start>')
    
    #for keeping track of variables
    variables = []
    var_count=0
    
    for token in tokens:
        
        if token.type not in [0,57,58,59,60,61,62,63,256]:
            #print('----')
            #print(token.type, token.string)
            #print(token)
            
            #keyword or variable
            if token.type == 1 and genaraic_vars:
                
                #key word
                if token.string in key_words:
                    parsed.append(token.string)
                
                #variable
                else:
                    #new var
                    if token.string not in variables:
                        var_count+=1
                        parsed.append('var_'+str(var_count))
                        variables.append(token.string)
                    else:
                        parsed.append('var_'+str(variables.index(token.string)))

            
            #string
            elif token.type == 2:
                parsed.append('<number>')
                
            #number
            elif token.type == 3:
                parsed.append('<string>')
            
            #everything else
            else:
                parsed.append(token.string)
                
                
    parsed.append('<end>')
    return parsed

#EXAMPLE
num = 3
#'''
print('Original:')
print(code_snippets[num])
print(len(code_snippets[num]))
print()
print('Tokenized:')
print(tokenize_python(code_snippets[num], genaraic_vars=False ))
print('number of tokens:',len(tokenize_python(code_snippets[num])))
#'''

Original:
  def _create_file_if_needed(self):
    
    if not os.path.exists(self._file.filename()):
      old_umask = os.umask(0177)
      try:
        open(self._file.filename(), 'a+b').close()
      finally:
        os.umask(old_umask)


230

Tokenized:
['<start>', '  ', 'def', '_create_file_if_needed', '(', 'self', ')', ':', '\n', '    ', 'if', 'not', 'os', '.', 'path', '.', 'exists', '(', 'self', '.', '_file', '.', 'filename', '(', ')', ')', ':', '\n', '      ', 'old_umask', '=', 'os', '.', 'umask', '(', '<number>', '<number>', ')', '\n', 'try', ':', '\n', '        ', 'open', '(', 'self', '.', '_file', '.', 'filename', '(', ')', ',', '<string>', ')', '.', 'close', '(', ')', '\n', '', 'finally', ':', '\n', '        ', 'os', '.', 'umask', '(', 'old_umask', ')', '\n', '', '', '', '', '<end>']
number of tokens: 77


In [58]:
#small_code = code_snippets[:10000]
#small_comment =  comments[:10000]
small_code = code_snippets
small_comment =  comments

In [25]:
import nltk
#nltk.download('punkt')

In [ ]:
collections.Counter()

In [44]:
def get_valid_pairs_and_vocabs(small_comment,small_code):


    Source_Sequences = [] #eng_sent = []
    Target_Sequences = [] #fra_sent = []
    Source_Tokens = set() #eng_chars = set()
    Target_Tokens = set() #fra_chars = set()


    errors = 0
    for i in tqdm(range(len(small_code))):
        try:
            Eng = ''.join(ch for ch in small_comment[i] if ch.isalnum() or ch ==' ').lower()
            Eng_toks = nltk.word_tokenize(Eng)
            Target_Sequences.append(Eng_toks)

            python_toks = tokenize_python(small_code[i], genaraic_vars=False)
            Source_Sequences.append(python_toks)


            for nodeType in python_toks:
                if (nodeType not in Source_Tokens):
                    Source_Tokens.add(nodeType)


            for word in Eng_toks:
                if (word not in Target_Tokens):
                    Target_Tokens.add(word)
        except Exception as e:
            #print(e)
            errors+=1

    print('number of errors:',errors,'or approixmatiely ',(errors/len(small_code))*100,'% of samples')
    return Source_Sequences,Target_Sequences, Source_Tokens, Target_Tokens
Source_Sequences,Target_Sequences, Source_Tokens, Target_Tokens = get_valid_pairs_and_vocabs(small_comment,small_code)

100%|██████████| 2508330/2508330 [17:40<00:00, 2364.83it/s] 


number of errors: 74483 or approixmatiely  2.9694258729911933 % of samples


In [45]:
Source_Tokens = sorted(list(Source_Tokens))
Target_Tokens = sorted(list(Target_Tokens))
nb_samples= len(Source_Sequences) #for all

# dictionary to index each english character - key is index and value is english character
source_index_to_token_dict = {}

# dictionary to get english character given its index - key is english character and value is index
source_token_to_index_dict = {}

for k, v in enumerate(Source_Tokens):
    source_index_to_token_dict[k] = v
    source_token_to_index_dict[v] = k

# dictionary to index each french character - key is index and value is french character
target_index_to_token_dict = {}

# dictionary to get french character given its index - key is french character and value is index
target_token_to_index_dict = {}

for k, v in enumerate(Target_Tokens):
    target_index_to_token_dict[k] = v
    target_token_to_index_dict[v] = k
    
    
max_len_source_seq = max([len(line) for line in Source_Sequences])
max_len_target_seq = max([len(line) for line in Target_Sequences])

print('max length source: ',max_len_source_seq)
print('source vocab: ',len(Source_Tokens))
print('max length target: ',max_len_target_seq)
print('target vocab: ',len(Target_Tokens))


max length source:  638
source vocab:  2698909
max length target:  190
target vocab:  1655801


In [69]:
import collections
snippets = code_snippets
tokens_counter = collections.Counter()
for snip in tqdm(snippets):
    try:
        tokens = tokenize_python(snip, genaraic_vars=False)
        for token in tokens:
            tokens_counter[token]+=1
    except:
        pass

100%|██████████| 2508330/2508330 [10:31<00:00, 3971.43it/s]


In [71]:
tokens_counter.most_common(10000)

[('\n', 15218542),
 (')', 12465233),
 ('(', 12465229),
 ('.', 11732459),
 (',', 8571487),
 ('', 6992687),
 ('=', 6595663),
 (':', 6484081),
 ('self', 6334728),
 ('<string>', 5003973),
 ('def', 2581619),
 ('<number>', 2508673),
 ('<start>', 2433847),
 ('<end>', 2433847),
 ('[', 2333323),
 (']', 2333320),
 ('        ', 2321817),
 ('    ', 2319294),
 ('return', 1814154),
 ('if', 1416531),
 ('            ', 1325266),
 ('None', 816003),
 ('in', 808511),
 ('for', 595680),
 ('+', 469270),
 ('class', 451918),
 ('==', 434459),
 ('not', 429623),
 ('name', 422450),
 ('\r\n', 410210),
 ('                ', 391755),
 ('*', 377103),
 ('else', 365466),
 ('{', 350714),
 ('}', 350713),
 ('True', 336296),
 ('-', 317166),
 ('data', 311143),
 ('path', 295640),
 ('False', 290264),
 ('value', 279809),
 ('%', 275028),
 ('raise', 273395),
 ('**', 265236),
 ('args', 264817),
 ('__init__', 257247),
 ('is', 248514),
 ('x', 243680),
 ('len', 242239),
 ('kwargs', 241433),
 ('i', 236598),
 ('get', 212608),
 ('os', 